<a href="https://colab.research.google.com/github/deethereal/NN/blob/main/NLP/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, pandas as pd
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

onehots={}
onehots['cat']=np.array([1,0,0,0])
onehots['the']=np.array([0,1,0,0])
onehots['dog']=np.array([0,0,1,0])
onehots['sat']=np.array([0,0,0,1])

sentence=['the','cat','sat','sat','dog']

x=onehots[sentence[0]]
for i in range(1,len(sentence)):
  x+=onehots[sentence[i]]
print(str(x))

[1 1 1 2]


In [ ]:
#lines=csv.DictReader("/content/gdrive/My Drive/IMDB.csv/",delimiter=',')
%cd "/content/drive/My Drive/"
lines=pd.read_csv('IMDB.csv', sep=',')
for line in lines:
  raw_reviews, raw_labels= lines['review'], lines['sentiment']
tokens=list(map(lambda x:set(x.split(" ")),raw_reviews))
vocab=set()
for sent in tokens:
  for word in sent:
    if len(word)>0:
      vocab.add(word)
vocab=list(vocab)
word2index={}
for i, word in enumerate(vocab):
  word2index[word]=i
input_dataset=list()
for sent in tokens:
  sent_indices = list()
  for word in sent:
    try:
      sent_indices.append(word2index[word])
    except:
      ""
  input_dataset.append(list(set(sent_indices)))
target_dataset=list()
for label in raw_labels:
  if label == 'positive':
    target_dataset.append(1)
  else:
    target_dataset.append(0)


/content/drive/My Drive


In [ ]:
print(len(vocab))

439838


In [ ]:
print(len(input_dataset[2]),len(raw_reviews[2].split()))
print(raw_reviews[2])

121 166
I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.


In [ ]:
S=len(input_dataset)
print(S)

50000


In [ ]:
np.random.seed(1)
def sigmoid(x):
  return 1/(1+np.exp(-x))
learning_rate, niter, hidden_size= 0.01, 35, 150
W01=0.2*np.random.random((len(vocab),hidden_size))-0.1
W12=0.2*np.random.random((hidden_size,1))-0.1
correct, total = 0,0
for iter in range(niter):
  for i in range(len(input_dataset)-2000):
    x,y=(input_dataset[i],target_dataset[i])
    l1=sigmoid(np.sum(W01[x],axis=0))
    l2=sigmoid(np.dot(l1,W12))
  

    l2_delta=l2-y
    l1_delta=l2_delta.dot(W12.T)

    W01[x]-=l1_delta*learning_rate
    W12-=np.outer(l1,l2_delta)*learning_rate

    if (np.abs(l2_delta) < 0.5):
      correct+=1
    total+=1
    """if (i% 10 == 9):
      progress=str(i/float(len(input_dataset)-26000))
      print("Iter: {} Progress: {}.{} TrainingAccuracy: {}".format(iter,progress[2:4],progress[4:6],(correct/float(total))))"""
correct, total = 0,0
for i in range(len(input_dataset)-2000,len(input_dataset)):
  x,y=(input_dataset[i],target_dataset[i])
  l1=sigmoid(np.sum(W01[x],axis=0))
  l2=sigmoid(np.dot(l1,W12))
    
  if np.abs(l2-y)<0.1:
    correct+=1
  total+=1
print('Test Accuracy: {}%'.format(100*correct/float(total)))

Test Accuracy: 81.75%


In [ ]:
def nn(x):
  l1=sigmoid(np.sum(W01[x],axis=0))
  l2=sigmoid(np.dot(l1,W12))
  print(l2)
  print("The nature of review is", end=" ")
  print("positive") if l2>0.5 else print("negative")
def proccecing(x):
  x=list(set(x.split(" ")))
  sent_indices = list()
  for word in x:
    try:
      sent_indices.append(word2index[word])
    except:
      ""
  return list(set(sent_indices))



In [ ]:
mr1="The film is very boring, I kept waiting for it to start !"
nn(proccecing(mr1))
mr2="I really liked this film, almost everything is great in it: the acting, the plot and the special effects !"
nn(proccecing(mr2))



[0.00367403]
The nature of review is negative
[0.99998366]
The nature of review is positive


***Эта архитектура ищет прямую корреляцию между входом и выходом***  
Поэтому получается так:

In [ ]:
test1='good film not bad'
test2='bad film not good'
nn(proccecing(test1))
nn(proccecing(test2))

[3.11175849e-05]
The nature of review is negative
[3.11175849e-05]
The nature of review is negative


In [ ]:
from collections import Counter
def similiar(target='beautiful'):
  target_index=word2index[target]
  scores=Counter()
  for word, index in word2index.items():
    raw_diff=W01[index]-W01[target_index]
    sqr=raw_diff*raw_diff
    scores[word]=-np.sqrt(sum(sqr))
  return scores.most_common(10)
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [ ]:
print(similiar('Marvel'))

[('Marvel', -0.0), ('ledger:', -0.37299359811339494), ('Marine(Naval', -0.37590073475230473), ('Peirce', -0.37950251177209005), ('"Heaven\'s', -0.37995265294910785)]


In [ ]:
np.random.seed(1)
%cd "/content/drive/My Drive/"
lines=pd.read_csv('IMDB.csv', sep=',')
for line in lines:
  raw_reviews, raw_labels= lines['review'], lines['sentiment']
tokens=list(map(lambda x:set(x.split(" ")),raw_reviews))
wordcnt= Counter() 
for sent in tokens:
  for word in sent:
    wordcnt[word]-=1
vocab=list(set(map(lambda x:x[0],wordcnt.most_common())))


word2index={}
for i,word in enumerate(vocab):
  word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
  sent_indices=list()
  for word in sent:
    try:
      sent_indices.append(word2index[word])
      concatenated.append(word2index[word])
    except:
      ""
  input_dataset.append(sent_indices)
concatenated=np.array(concatenated)

np.random.shuffle(input_dataset)
print("Data is ready!")

/content/drive/My Drive
Data is ready!


In [ ]:
learning_rate, hidden_size, niter = 0.05,50,5
window,negative = 2,5

W01=0.2*(np.random.random((len(vocab),hidden_size))-0.5)
W12=0.2*(np.random.random((len(vocab),hidden_size,)) -0.5)
l2_target=np.zeros(negative+1)
l2_target[0]=1
progress_l=-1


for rev_i, review in enumerate(input_dataset*niter):
  for target_i in range(len(review)):
    target_sample=[review[target_i]]+list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])  #выбираем случайное подмонжество


    left_context=review[max(0,target_i-window):target_i]
    right_context=review[target_i+1:min(len(review),target_i+window)]


    l1=np.mean(W01[left_context+right_context], axis=0)
    l2=sigmoid(l1.dot(W12[target_sample].T))

    l2_delta=l2-l2_target
    l1_delta=l2_delta.dot(W12[target_sample])

    W01[left_context+right_context]-=l1_delta*learning_rate
    W12[target_sample]-=np.outer(l2_delta,l1)*learning_rate
  current_p=round(rev_i/float(len(input_dataset)),1)  
  if progress_l<current_p:
    print('Progress: {} '.format(current_p))
    progress_l=current_p 
print(similiar('terrible'))


Progress: 0.0 
Progress: 0.1 
Progress: 0.2 
Progress: 0.3 
Progress: 0.4 
Progress: 0.5 
Progress: 0.6 
Progress: 0.7 
Progress: 0.8 
Progress: 0.9 
Progress: 1.0 
Progress: 1.1 
Progress: 1.2 
Progress: 1.3 
Progress: 1.4 
Progress: 1.5 
Progress: 1.6 
Progress: 1.7 
Progress: 1.8 
Progress: 1.9 
Progress: 2.0 
Progress: 2.1 
Progress: 2.2 
Progress: 2.3 
Progress: 2.4 
Progress: 2.5 
Progress: 2.6 
Progress: 2.7 
Progress: 2.8 
Progress: 2.9 
Progress: 3.0 
Progress: 3.1 
Progress: 3.2 
Progress: 3.3 
Progress: 3.4 
Progress: 3.5 
Progress: 3.6 
Progress: 3.7 
Progress: 3.8 
Progress: 3.9 
Progress: 4.0 
Progress: 4.1 
Progress: 4.2 
Progress: 4.3 
Progress: 4.4 
Progress: 4.5 
Progress: 4.6 
Progress: 4.7 
Progress: 4.8 
Progress: 4.9 
Progress: 5.0 
[('terrible', -0.0), ('rival', -4.199290883535479), ('sister.', -4.508783023488256), ('Except', -4.545213670036982), ('assuming', -4.546449340962891), ('guys.', -4.624140835325655), ('So', -4.764357390823417), ('average.', -4.842264849

In [ ]:
print(similiar('good'))

[('good', -0.0), ('video,', -6.514949966016076), ('doubt,', -6.759650566502447), ('intention', -6.8007262864054345), ('contribution', -6.811713649884149), ('/>John', -6.859748670258398), ('mess,', -6.95821079913052), ('documentary', -7.022108610069692), ('river', -7.080200197335381), ('passing', -7.083841749468614)]


In [ ]:
def analogy(pos=[ 'terrible','good' ], negative = ['bad'] ):
  norms=np.sum(W01*W01,axis=1)
  norms.resize(norms.shape[0],1)

  normed_W=W01*norms
  query_vect = np.zeros(len(W01[0]))
  for word in pos:
    query_vect+=normed_W[word2index[word]]
  for word in negative:
    query_vect-=normed_W[word2index[word]]
  scores = Counter()
  for word, index in word2index.items():
    raw_diff= W01[index]-query_vect
    sqr=raw_diff*raw_diff
    scores[word]=np.sqrt(sum(sqr))
  return scores.most_common(10)[1:]
print(analogy())

[('sense', 1097.1943195290353), ('masterpiece', 1097.1491971455425), ('ordinary', 1097.0758389074565), ('tension.', 1097.0502683700718), ('relief', 1096.9729424463587), ('ultimate', 1096.9243475543674), ('genius.', 1096.8958663933558), ('Picture', 1096.8765726437382), ('boys.', 1096.8714072843113)]


In [2]:
import numpy as np
def softmax(x_):
  x=np.atleast_2d(x_)
  temp=np.exp(x)
  return temp/np.sum(temp,axis=1,keepdims=True)
word_vects={}
word_vects['yankes']=np.array([[0.,0.,0.]])
word_vects['bears']=np.array([[0.,0.,0.]])
word_vects['braves']=np.array([[0.,0.,0.]])
word_vects['red']=np.array([[0.,0.,0.]])
word_vects['sox']=np.array([[0.,0.,0.]])
word_vects['lose']=np.array([[0.,0.,0.]])
word_vects['defeat']=np.array([[0.,0.,0.]])
word_vects['beat']=np.array([[0.,0.,0.]])
word_vects['tie']=np.array([[0.,0.,0.]])

sent2output=np.random.rand(3,len(word_vects))

identity=np.eye(3)

In [4]:
l0=word_vects['red']
l1=l0.dot(identity)+word_vects['sox']
l2=l1.dot(identity)+word_vects['defeat']

pred=softmax(l2.dot(sent2output))
print(pred)

[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


In [ ]:
yankes=bp.array([1,0,0,0,0,0,0,0,0])
pred_delta=pred-y
l2_delta=pred_delta.dot(sent2output.T)
defeat_delta=l2_delta
l1_delta=l2_delta.dot(identity.T)
sox_delta=l1_delta
l0_delta=l1_delta.dot(identity.T)
lr=0.01
word_vects['red']-=l0_delta*lr 
word_vects['sox']-=sox_delta*lr 
word_vects['defeat']-=defeat_delta*lr 
identity-=np.outer(l0,l1_delta)*lr 
identity-=np.outer(l1,l2_delta)*lr 
sent2output-=np.outer(l2,pred_delta) *lr 


In [ ]:
!wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-1.tar.gz
!tar -xvf tasks_1-20_v1-1.tar.gz

In [22]:
import sys,random,math
from collections import Counter
import numpy as np
f=open('tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw=f.readlines()
f.close()
tokens=list()
for line in raw[0:1000]:
  tokens.append(line.lower().replace('\n','').replace('?','').replace('\t','').split(' ')[1:])
for t in tokens:
  while t[3][-1] in ['1','2','3','4','5','6','7','8','9']:
    t[3]=t[3][:-1]
for i in range(0,33,3):
  print(tokens[i],tokens[i+1],tokens[i+2])

['mary', 'moved', 'to', 'the', 'bathroom.'] ['john', 'went', 'to', 'the', 'hallway.'] ['where', 'is', 'mary', 'bathroom']
['daniel', 'went', 'back', 'to', 'the', 'hallway.'] ['sandra', 'moved', 'to', 'the', 'garden.'] ['where', 'is', 'daniel', 'hallway']
['john', 'moved', 'to', 'the', 'office.'] ['sandra', 'journeyed', 'to', 'the', 'bathroom.'] ['where', 'is', 'daniel', 'hallway']
['mary', 'moved', 'to', 'the', 'hallway.'] ['daniel', 'travelled', 'to', 'the', 'office.'] ['where', 'is', 'daniel', 'office']
['john', 'went', 'back', 'to', 'the', 'garden.'] ['john', 'moved', 'to', 'the', 'bedroom.'] ['where', 'is', 'sandra', 'bathroom']
['sandra', 'travelled', 'to', 'the', 'office.'] ['sandra', 'went', 'to', 'the', 'bathroom.'] ['where', 'is', 'sandra', 'bathroom']
['mary', 'went', 'to', 'the', 'bedroom.'] ['daniel', 'moved', 'to', 'the', 'hallway.'] ['where', 'is', 'sandra', 'bathroom']
['john', 'went', 'to', 'the', 'garden.'] ['john', 'travelled', 'to', 'the', 'office.'] ['where', 'is', 

In [24]:
vocab=set()
for sent in tokens:
  for word in sent:
    vocab.add(word)
vocab=list(vocab)
word2index={}
for i,word in enumerate(vocab):
  word2index[word]=i
def word2indices(sentence):
  idx=list()
  for word in sentence:
    idx.append(word2index[word])
  return idx
def softmax(x):
  e_x=np.exp(x-np.max(x))
  return e_x/e_x.sum(axis=0)

In [37]:
def predict(sent):
  layers=list()
  layer={}
  layer['hidden']=start
  layers.append(layer)
  loss=0
  preds=list()
  for target_i in range(len(sent)):
    layer={}
    layer['pred']=softmax(layers[-1]['hidden'].dot(decoder))
    loss+=-np.log(layer['pred'][sent[target_i]])
    layer['hidden']=layers[-1]['hidden'].dot(reccurent)+embed[sent][target_i]
    layers.append(layer)
  return layers,loss


In [52]:
np.random.seed(1)
embed_size=10
embed=(np.random.rand(len(vocab),embed_size)-0.5)*0.1
reccurent=np.eye(embed_size)
start=np.zeros(embed_size)
decoder=(np.random.rand(embed_size,len(vocab))-0.5)*0.1
one_hot = np.eye(len(vocab))
learning_rate=0.01
for iter in range(38000):
  sent=word2indices(tokens[iter%len(tokens)][1:])
  layers,loss=predict(sent)
  for layer_idx in reversed(range(len(layers))):
    layer=layers[layer_idx]
    target = sent[layer_idx-1]
    if (layer_idx>0):
      layer['output_delta']=layer['pred']-one_hot[target]
      new_hidden_delta=layer['output_delta'].dot(decoder.T)
      if (layer_idx==len(layers)-1):
        layer['hidden_delta']=new_hidden_delta
      else:
        layer['hidden_delta']=new_hidden_delta+layers[layer_idx+1]['hidden_delta'].dot(reccurent.T)
    else:
      layer['hidden_delta']=layers[layer_idx+1]['hidden_delta'].dot(reccurent.T)    
  start-=layers[0]['hidden_delta']*learning_rate/float(len(sent))
  for layer_idx, layer in enumerate(layers[1:]):
    decoder-=np.outer(layers[layer_idx]['hidden'],layer['output_delta'])*learning_rate/float(len(sent))
    embed_idx=sent[layer_idx] 
    embed[embed_idx]-=layers[layer_idx]['hidden_delta']*learning_rate/float(len(sent))
    reccurent-=np.outer(layers[layer_idx]['hidden_delta'], layer['hidden_delta'])*learning_rate/float(len(sent))
  if iter%1000 ==0:
    print("Перплексия: " +str(np.exp(loss/len(sent))))   



Перплексия: 30.963606957387032
Перплексия: 27.71866579607081
Перплексия: 24.595649659970682
Перплексия: 21.724290570494933
Перплексия: 18.246991785631984
Перплексия: 13.948758743495924
Перплексия: 9.138576006070066
Перплексия: 8.039250038607824
Перплексия: 7.911053190462284
Перплексия: 8.572291615523767
Перплексия: 10.471806460466208
Перплексия: 10.998395084030497
Перплексия: 9.489275387261065
Перплексия: 7.79456714675163
Перплексия: 6.156219912462877
Перплексия: 5.752824356611411
Перплексия: 6.953222861265637
Перплексия: 7.378860344340722
Перплексия: 6.258279108279967
Перплексия: 5.211686625632706
Перплексия: 5.3091401158635305
Перплексия: 5.825962511525318
Перплексия: 6.4187292737531685
Перплексия: 6.672027494036467
Перплексия: 6.357797371479096
Перплексия: 5.827879574224959
Перплексия: 5.364145952938768
Перплексия: 5.247328339466191
Перплексия: 5.363271149647524
Перплексия: 5.753115906033597
Перплексия: 6.081939027861756
Перплексия: 5.361506250765158
Перплексия: 4.514980863514879
Пе

In [53]:
sent_index=4
l,_=predict(word2indices(tokens[sent_index]))
print(tokens[sent_index])
for i,each_layer in enumerate(l[1:-1]):
  input=tokens[sent_index][i]
  true=tokens[sent_index][i+1]
  pred=vocab[each_layer['pred'].argmax()]
  print('Предыдущее слово: '+input+(" "*(12-len(input)))+"Ожидание: "+true+(" "*(15-len(true))) + "Предсказание: "+pred )

['sandra', 'moved', 'to', 'the', 'garden.']
Предыдущее слово: sandra      Ожидание: moved          Предсказание: is
Предыдущее слово: moved       Ожидание: to             Предсказание: to
Предыдущее слово: to          Ожидание: the            Предсказание: to
Предыдущее слово: the         Ожидание: garden.        Предсказание: the
